In [ ]:
import os
import cv2
import math


def slice_image_with_labels(img_path, labels_path, output_dir="output", tile_size=320):
    images_dir = os.path.join(output_dir, "images")
    labels_dir = os.path.join(output_dir, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    img = cv2.imread(img_path)
    if img is None:
        print(f" Nie udało się wczytać {img_path}")
        return
    h, w, _ = img.shape
    basename = os.path.splitext(os.path.basename(img_path))[0]
    labels = []
    if labels_path and os.path.exists(labels_path):
        with open(labels_path, "r") as f:
            for line in f.readlines():
                cls, xc, yc, bw, bh = map(float, line.strip().split())
                labels.append([int(cls), xc * w, yc * h, bw * w, bh * h])
    nx = math.ceil(w / tile_size)
    ny = math.ceil(h / tile_size)
    for i in range(nx):
        for j in range(ny):
            x1 = min(i * tile_size, w - tile_size)
            y1 = min(j * tile_size, h - tile_size)
            x2 = x1 + tile_size
            y2 = y1 + tile_size
            tile = img[y1:y2, x1:x2]
            out_name = f"{basename}_{j}_{i}"
            out_img_path = os.path.join(images_dir, out_name + ".jpg")
            out_label_path = os.path.join(labels_dir, out_name + ".txt")
            cv2.imwrite(out_img_path, tile)
            new_labels = []
            for cls, xc, yc, bw, bh in labels:
                x_min = xc - bw / 2
                y_min = yc - bh / 2
                x_max = xc + bw / 2
                y_max = yc + bh / 2
                if x_max <= x1 or x_min >= x2 or y_max <= y1 or y_min >= y2:
                    continue
                new_xmin = max(x_min, x1)
                new_ymin = max(y_min, y1)
                new_xmax = min(x_max, x2)
                new_ymax = min(y_max, y2)
                new_xmin -= x1
                new_xmax -= x1
                new_ymin -= y1
                new_ymax -= y1
                bw_new = new_xmax - new_xmin
                bh_new = new_ymax - new_ymin
                xc_new = new_xmin + bw_new / 2
                yc_new = new_ymin + bh_new / 2
                xc_norm = xc_new / tile_size
                yc_norm = yc_new / tile_size
                bw_norm = bw_new / tile_size
                bh_norm = bh_new / tile_size
                if bw_norm > 0 and bh_norm > 0:
                    new_labels.append(
                        f"{cls} {xc_norm:.6f} {yc_norm:.6f} {bw_norm:.6f} {bh_norm:.6f}"
                    )
            with open(out_label_path, "w") as f:
                f.write("\n".join(new_labels))
    print(f"Podzielono {img_path}, zapisano do {output_dir}")


def process_path_with_labels(path, labels_dir, output_dir="output", tile_size=320):
    if os.path.isfile(path):
        base = os.path.splitext(os.path.basename(path))[0]
        label_path = os.path.join(labels_dir, base + ".txt")
        slice_image_with_labels(path, label_path, output_dir, tile_size)
    elif os.path.isdir(path):
        for file in os.listdir(path):
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                base = os.path.splitext(file)[0]
                label_path = os.path.join(labels_dir, base + ".txt")
                slice_image_with_labels(
                    os.path.join(path, file), label_path, output_dir, tile_size
                )
    else:
        print("Podana ścieżka nie istnieje")


process_path_with_labels
(
    r"C:\Users\marty\histolab_project\dane_do_treningu\dane_pomieszane\polaczone\rozmiar_początkowy\test\images",
    r"C:\Users\marty\histolab_project\dane_do_treningu\dane_pomieszane\polaczone\rozmiar_początkowy\test\labels",
    r"C:\Users\marty\histolab_project\dane_do_treningu\dane_komorki\trening\Nowy folder",
    320,
)

In [ ]:
from pathlib import Path
import pandas as pd

# 21_09830A1_19_ROI_0_0_5

TXT_PATH = r"C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Dane\CD34_tiles_yolo\labels_pełne"
SECTION_DIMS = (130, 130)
ROI_DESC = r"..\data\adding_xmls\ROI_zestaw2.csv"
OUT_PATH = r"..\data\adding_xmls"

txt_dir = Path(TXT_PATH)
count = sum(1 for f in txt_dir.iterdir() if f.is_file() and f.suffix == ".txt")
print(f"Number of .txt files: {count}")

df = pd.read_csv(ROI_DESC)
# df = df[(df["slide"] == "21_09830A1_19")]
# print(df[(df["roi_id"] == 1) & (df["slide"] == "22_09617A1_7")].iloc[0].x_min)

out_data = {"x_min": [], "y_min": [], "width": [], "height": [], "filename": []}#, "wsi": [], "roi_id":[], "section_j": [], "section_i": []}

for file in txt_dir.iterdir():
    filename_elems = file.name.split(".")[0].split("_")

    wsi = "_".join(filename_elems[:3])
    roi_id = int(filename_elems[4])
    section_y_idx = int(filename_elems[-2])
    section_x_idx = int(filename_elems[-1])

    print(roi_id, "-->", file)
    roi_data = df[(df["slide"] == wsi) & (df["roi_id"] == roi_id)]

    print(len(roi_data))
    if len(roi_data) < 1:
        continue

    roi_data = roi_data.iloc[0]
    
    roi_x_max = roi_data.x_min + roi_data.width
    roi_y_max = roi_data.y_min + roi_data.height

    section_x_min = min(roi_data.x_min + section_x_idx * SECTION_DIMS[0], roi_x_max - SECTION_DIMS[0])
    section_y_min = min(roi_data.y_min + section_y_idx * SECTION_DIMS[1], roi_y_max - SECTION_DIMS[1])

    print(roi_data.y_min, section_y_idx, section_y_min)

    out_data["x_min"].append(section_x_min)
    out_data["y_min"].append(section_y_min)
    out_data["width"].append(SECTION_DIMS[0])
    out_data["height"].append(SECTION_DIMS[1])
    out_data["filename"].append(file.name)
    # out_data["roi_id"].append(roi_id)
    # out_data["wsi"].append(wsi)
    # out_data["section_i"].append(sec)
    # out_data["section_j"].append(roi_id)

out_df = pd.DataFrame(data=out_data)
out_df.to_csv(
    f"{OUT_PATH}\\yolo_section_descriptions_zestaw2_5.csv",
    index_label="section_id",
)